In [6]:
## Imports
# Data processing
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity

In [24]:
# Read datasets

users_colnames=['userId', 'gender', 'age', 'profession']
movies_colnames=['movieId', 'year', 'title']
ratings_colnames=['userId', 'movieId', 'rating']

users = pd.read_csv('cse2525-reccommender-systems-challenge/users.csv', sep=';', names=users_colnames)
movies = pd.read_csv('cse2525-reccommender-systems-challenge/movies_v2.csv', sep=';', names=movies_colnames) # changed line 3601 from dataset : ';' -> '_'
ratings = pd.read_csv('cse2525-reccommender-systems-challenge/ratings.csv', sep=';', names=ratings_colnames)

# Check
users.head()
movies.head()
ratings.head()


,movieId,year,title
0,1,1995,Toy_Story_(1995)
1,2,1995,Jumanji_(1995)
2,3,1995,Grumpier_Old_Men_(1995)
3,4,1995,Waiting_to_Exhale_(1995)
4,5,1995,Father_of_the_Bride_Part_II_(1995)


In [50]:
# Count the # of ratings per movie
ratings_by_movie_id = ratings.groupby(['movieId'])['movieId']\
    .count()\
    .reset_index(name='counts')

ratings_by_movie_id.head()


,movieId,counts
0,1,1896
1,2,635
2,3,443
3,4,155
4,5,270


In [53]:
# Number of rows remaining after saving only the ones that have `>= threshold` ratings
movies_threshold = 100
len(ratings_by_movie_id[ratings_by_movie_id['counts'] >= movies_threshold])


1940